你已经构建了一个模型。但它有多好呢？

在本课程中，您将学习使用模型验证来衡量您的模型的质量。衡量模型质量是迭代改进模型的关键。

# 什么是模型验证？

您几乎需要评估您构建的每个模型。在大多数（但不是全部）应用程序中，模型质量的相关度量标准是预测准确性。换句话说，模型的预测是否接近实际发生的情况。

许多人在测量预测准确性时犯了一个严重的错误。他们使用 *训练数据* 进行预测，并将这些预测与 *训练数据* 中的目标值进行比较。您将在接下来的内容中了解到这种方法的问题以及如何解决问题，但首先让我们考虑如何进行评估。

您首先需要将模型质量概括成一个可理解的指标。如果您比较了10,000个房屋的预测和实际价值，您可能会发现好的和坏的预测混杂在一起。查看10,000个预测和实际值的列表是没有意义的。我们需要将其总结为单一指标。

有许多评估模型质量的指标，但我们将从一个称为 **平均绝对误差**（也称为 **MAE**）的指标开始。让我们从最后一个单词“误差”开始解释这个指标。

每个房屋的预测误差是：<br>

```
error=actual−predicted
```

因此，如果一栋房子的成本为150,000美元，而预测的成本为100,000美元，误差为50,000美元。

使用MAE指标，我们取每个误差的绝对值。这将每个误差转换为正数。然后，我们取这些绝对误差的平均值。这是我们的模型质量测量指标。用简单的语言来说，可以表示为

> 平均而言，我们的预测误差大约为X。

计算MAE，我们首先需要一个模型。该模型建立在下面的隐藏单元格中，您可以通过点击`code`按钮进行审核。

In [1]:
# 隐藏在这里的数据加载代码
import pandas as pd

# Load data
melbourne_file_path = '../input/melbourne-housing-snapshot/melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path) 
# Filter rows with missing price values
filtered_melbourne_data = melbourne_data.dropna(axis=0)
# Choose target and features
y = filtered_melbourne_data.Price
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'BuildingArea', 
                        'YearBuilt', 'Lattitude', 'Longtitude']
X = filtered_melbourne_data[melbourne_features]

from sklearn.tree import DecisionTreeRegressor
# Define model
melbourne_model = DecisionTreeRegressor()
# Fit model
melbourne_model.fit(X, y)

DecisionTreeRegressor()

一旦我们有了一个模型，下面是我们如何计算平均绝对误差:

In [2]:
from sklearn.metrics import mean_absolute_error

predicted_home_prices = melbourne_model.predict(X)
mean_absolute_error(y, predicted_home_prices)

434.71594577146544

# “样本内”分数的问题

我们刚刚计算的度量可以称为“样本内”分数。我们使用同一“样本”房屋来建立模型和评估模型。以下是此方法的弊端。

假设在大房地产市场中，门的颜色与房价无关。

但是，在您用于建立模型的数据样本中，所有具有绿色门的房屋都非常昂贵。模型的任务是找到预测房价的模式，因此它将看到这个模式，并且它将始终预测绿色门的房屋价格很高。

由于这种模式是从训练数据中推导出来的，因此模型在训练数据中看起来准确无误。

但是，如果该模式在模型看到新数据时不成立，则该模型在实践中会非常不准确。

由于模型的实际价值来自于对新数据进行预测，因此我们在未用于构建模型的数据上测量性能。最简单的方法是从建模过程中排除一些数据，然后使用这些数据来测试模型在其从未看过的数据上的准确性。这些数据称为 **验证数据**。

# 编码它

scikit-learn库具有一个名为`train_test_split`的函数，可以将数据分成两部分。我们将使用其中一部分数据作为训练数据来拟合模型，而将其他数据作为验证数据来计算`平均绝对误差`。

以下是代码：

In [3]:
from sklearn.model_selection import train_test_split

# 为特征和目标将数据分割为训练和验证数据
# 分割是基于一个随机数生成器。
# 为 Random_state 参数提供一个数值可以保证我们每次运行这个脚本时得到相同的分割。
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)
# 定义模型
melbourne_model = DecisionTreeRegressor()
# 拟合模型
melbourne_model.fit(train_X, train_y)

# 得到验证数据的预测价格
val_predictions = melbourne_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

258930.03550677857


# 哇！

样本内数据的平均绝对误差约为500美元，样本外数据的平均绝对误差超过25万美元。

这就是一个几乎完全正确的模型和一个不能用于大多数实际目的的模型之间的区别。作为参考，验证数据中的平均房屋价值是110万美元。因此，新数据的误差约为平均房屋价值的四分之一。

有很多方法可以改进这个模型，比如试验找到更好的特征或者不同的模型类型。

# 练习时间
在我们考虑改进这个模型之前，请自己尝试**[Model Validation](https://www.kaggle.com/kernels/fork/1259097)**

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/intro-to-machine-learning/discussion) to chat with other learners.*